# 第一章：設定你的 Go 環境

歡迎來到 Go 語言學習之旅！在這個章節中，我們將學習如何設定 Go 開發環境，並深入了解 Go 的依賴管理演進歷史。

## 學習目標
- 了解 Go 版本演進與重要里程碑
- 掌握 GOPATH 與 Go Modules 的差異與用途
- 理解 go.mod 和 go.sum 檔案的作用
- 熟悉 Go 命令在不同版本的行為差異
- 學會依賴管理的最佳實踐

## Go 版本演進與重要里程碑

Go 的依賴管理經歷了重大演進，了解這些變化對於理解現代 Go 開發至關重要：

### 📈 關鍵版本里程碑

| Go 版本 | 發布時間 | 重要變化 |
|---------|----------|----------|
| **Go 1.8** | ?? | • GOPATH預設為  ~/go |
| **Go 1.11** | 2018.08 | • 引入 Go Modules（實驗性功能）<br>• 新增 `GO111MODULE` 環境變數<br>• 可在 GOPATH 外建立專案 |
| **Go 1.13** | 2019.09 | • Go Modules 成為預設模式<br>• `GO111MODULE=on` 為預設<br>• 引入 `go.sum` checksum 驗證<br>• 新增 sumdb 校驗資料庫 |
| **Go 1.16** | 2021.02 | • 完全移除 `GO111MODULE=auto` 支援<br>• GOPATH 模式正式棄用<br>• `go install pkg@version` 新語法 |
| **Go 1.17+** | 2021.08+ | • `go get` 不再安裝程式到 GOBIN<br>• 改用 `go install` 安裝工具<br>• Module 圖解析優化 |
| **Go 1.20+** | 2023.02+ | • 效能改進與安全性增強<br>• Module 工作區模式改進 |

### 🔄 GO111MODULE 環境變數演進

- **`GO111MODULE=off`**: 強制使用 GOPATH 模式（已棄用）
- **`GO111MODULE=on`**: 強制使用 Go Modules 模式
- **`GO111MODULE=auto`**: 自動偵測（Go 1.16 後不再支援）

In [1]:
/* 檢查當前 Go 版本與環境設定 */
package main

import (
    "fmt"
    "os"
    "runtime"
)

func main() {
    fmt.Println("=== Go 版本與環境資訊 ===")
    
    // 檢查 Go 版本
    fmt.Printf("Go 版本: %s\n", runtime.Version())  // 顯示: go1.21.0
    fmt.Printf("作業系統: %s\n", runtime.GOOS)       // 顯示: darwin/linux/windows
    fmt.Printf("架構: %s\n", runtime.GOARCH)         // 顯示: amd64/arm64
    
    // 檢查重要環境變數
    fmt.Printf("GOROOT: %s\n", runtime.GOROOT())    // Go 安裝路徑
    
    // 檢查 GOPATH（可能為空）
    if gopath := os.Getenv("GOPATH"); gopath != "" {
        fmt.Printf("GOPATH: %s\n", gopath)
    } else {
        fmt.Println("GOPATH: 未設定（使用 Go Modules 模式）")
    }
    
    // 檢查 GO111MODULE（Go 1.16+ 可能已無此變數）
    if gomod := os.Getenv("GO111MODULE"); gomod != "" {
        fmt.Printf("GO111MODULE: %s\n", gomod)
    } else {
        fmt.Println("GO111MODULE: 未設定（Go 1.13+ 預設為 on）")
    }
}

=== Go 版本與環境資訊 ===
Go 版本: go1.24.5
作業系統: darwin
架構: arm64
GOROOT: /opt/homebrew/Cellar/go/1.24.5/libexec
GOPATH: /Users/hank/go
GO111MODULE: 未設定（Go 1.13+ 預設為 on）


## 1. GOPATH 模式（Go 1.11 之前的傳統模式）

GOPATH 是 Go Modules 出現前的核心概念，雖然已被取代，但理解它有助於了解 Go 的發展脈絡。

### 📁 GOPATH 目錄結構

預設路徑：
- **Linux/macOS**: `~/go`
- **Windows**: `%USERPROFILE%\go`

```
$GOPATH/
├── src/           # 專案原始碼與第三方套件 source code
│   ├── github.com/
│   │   └── user/project/
│   └── example.com/
│       └── another/project/
├── pkg/           # 編譯後的中間檔 (.a 檔案)
│   ├── mod/       # Go Modules 快取（Go 1.11+）
│   └── sumdb/     # checksum cache（Go 1.13+）
└── bin/           # 可執行檔 (go install 的輸出)
```

### 🔍 各目錄用途詳解

#### `src/` 目錄
- **舊模式**：所有 Go 專案都必須放在 `$GOPATH/src/<module>` 下
- **作用**：編譯器尋找 import 的 source code
- **Go Modules 後**：大多不再使用

#### `pkg/` 目錄
- **`mod/`**: module source code 快取，Go 1.11+
- **`sumdb/`**: 模組 checksum cache，Go 1.13+
- **功能**：加速編譯、驗證 module 不被篡改

#### `bin/` 目錄
- **儲存**：編譯後的 binary，可全域執行
- **生成**：透過 `go install` 指令

In [ ]:
/* 檢查 GOPATH 目錄結構 */
package main

import (
    "fmt"
    "os"
    "path/filepath"
)

func main() {
    fmt.Println("=== GOPATH 目錄結構檢查 ===")
    
    // 取得 GOPATH（如果未設定則使用預設值）
    gopath := os.Getenv("GOPATH")
    if gopath == "" {
        // 預設 GOPATH
        homeDir, _ := os.UserHomeDir()
        gopath = filepath.Join(homeDir, "go")
        fmt.Printf("GOPATH 未設定，使用預設路徑: %s\n", gopath)
    } else {
        fmt.Printf("GOPATH: %s\n", gopath)
    }
    
    // 檢查 GOPATH 的標準目錄
    dirs := []struct {
        name string
        desc string
    }{
        {"src", "原始碼目錄（GOPATH 模式）"},
        {"pkg", "編譯快取目錄"},
        {"pkg/mod", "Go Modules 快取（Go 1.11+）"},
        {"pkg/sumdb", "校驗和資料庫快取（Go 1.13+）"},
        {"bin", "可執行檔目錄"},
    }
    
    fmt.Println("\n📁 目錄結構:")
    for _, dir := range dirs {
        fullPath := filepath.Join(gopath, dir.name)
        if _, err := os.Stat(fullPath); err == nil {
            fmt.Printf("✅ %s/ - %s\n", dir.name, dir.desc)
        } else {
            fmt.Printf("❌ %s/ - %s (不存在)\n", dir.name, dir.desc)
        }
    }
    
    fmt.Println("\n💡 提醒: 現代 Go 開發主要使用 Go Modules，不依賴 GOPATH/src")
}

## 2. GOMODCACHE（Go Modules 專屬快取）

Go 1.11 引入 Go Modules 後，依賴管理有了全新的快取機制。

### 📦 GOMODCACHE 結構與特點

**預設路徑**：`$GOPATH/pkg/mod`（通常是 `~/go/pkg/mod`）

```
$GOMODCACHE/
├── github.com/gin-gonic/gin@v1.9.1/     # 特定版本的模組
├── github.com/gin-gonic/gin@v1.8.2/     # 可同時存在多版本
├── github.com/stretchr/testify@v1.8.4/
├── cache/                               # 下載快取 zip
└── download/                            # 下載中間檔案
```

### 🔍 關鍵差異：GOPATH vs GOMODCACHE

| 特性 | GOPATH 模式 | Go Modules 模式 |
|------|-------------|------------------|
| **專案位置** | 必須在 `$GOPATH/src` | 任何目錄都可以 |
| **依賴存放** | `$GOPATH/src/` | `$GOMODCACHE` |
| **版本管理** | 無版本控制 | 語義化版本控制 |
| **依賴隔離** | 全域共享 | 每個專案獨立 |
| **建置重現性** | 困難 | 透過 go.sum 保證 |
| **多版本支援** | 不支援 | 完全支援 |

### 🎯 GOMODCACHE 優勢
- **版本鎖定**: 精確控制依賴版本
- **快取機制**: 加速重複下載
- **安全驗證**: 透過 checksum 驗證完整性
- **空間效率**: 多專案共享相同版本的依賴

In [ ]:
/* 檢查 GOMODCACHE 位置與內容 */
package main

import (
    "fmt"
    "os"
    "os/exec"
    "path/filepath"
    "strings"
)

func main() {
    fmt.Println("=== GOMODCACHE 檢查 ===")
    
    // 使用 go env 命令檢查 GOMODCACHE 位置
    cmd := exec.Command("go", "env", "GOMODCACHE")
    output, err := cmd.Output()
    if err != nil {
        fmt.Printf("❌ 無法取得 GOMODCACHE: %v\n", err)
        return
    }
    
    gomodcache := strings.TrimSpace(string(output))
    fmt.Printf("📦 GOMODCACHE: %s\n", gomodcache)
    
    // 檢查目錄是否存在
    if _, err := os.Stat(gomodcache); err == nil {
        fmt.Println("✅ 模組快取目錄存在")
        
        // 檢查子目錄
        subDirs := []string{"cache", "download"}
        for _, subDir := range subDirs {
            subPath := filepath.Join(gomodcache, subDir)
            if _, err := os.Stat(subPath); err == nil {
                fmt.Printf("  ├── %s/ 存在\n", subDir)
            } else {
                fmt.Printf("  ├── %s/ 不存在\n", subDir)
            }
        }
    } else {
        fmt.Println("❌ 模組快取目錄不存在")
        fmt.Println("💡 執行 'go mod download' 後會自動建立")
    }
    
    // 比較 GOPATH 與 GOMODCACHE
    fmt.Println("\n🔍 重要概念:")
    fmt.Println("• GOPATH 模式: 依賴存放在 $GOPATH/src/")
    fmt.Println("• Go Modules: 依賴存放在 $GOMODCACHE")
    fmt.Println("• GOMODCACHE 支援多版本並存")
    fmt.Println("• 每個版本都有獨立的目錄")
}

## 3. go.mod 檔案：模組定義的核心

Go 1.11 引入的 `go.mod` 檔案是 Go Modules 的核心，用於定義模組資訊和依賴關係。

### 📄 go.mod 檔案結構

```go
module github.com/myuser/myproject    // 模組名稱

go 1.21                              // Go 版本要求

require (                            // 直接依賴
    github.com/gin-gonic/gin v1.9.1
    github.com/stretchr/testify v1.8.4
)

require (                            // 間接依賴（自動管理）
    github.com/bytedance/sonic v1.9.1 // indirect
    github.com/chenzhuoyu/base64x v0.0.0-20221115062448-fe3a3abad311 // indirect
)

replace (                            // 替換依賴（用於本地開發）
    github.com/old/package => github.com/new/package v1.0.0
    github.com/local/package => ../local-package
)

exclude (                            // 排除特定版本
    github.com/problematic/package v1.2.3
)
```

### 🏷️ go.mod 關鍵字說明

- **`module`**: 定義當前模組的導入路徑
- **`go`**: 指定最低 Go 版本要求
- **`require`**: 列出依賴模組和版本
- **`indirect`**: 標記間接依賴（自動添加）
- **`replace`**: 替換或重定向依賴
- **`exclude`**: 排除特定版本的依賴

### 🔄 go.mod 自動更新時機

以下操作會自動修改 `go.mod`：
- `go get package@version` - 添加新依賴
- `go mod tidy` - 清理未使用的依賴
- `go build` / `go run` - 解析缺失依賴

In [ ]:
/* 檢查 go.mod 檔案內容與模組資訊 */
package main

import (
    "fmt"
    "io/ioutil"
    "os"
    "path/filepath"
    "strings"
)

func main() {
    fmt.Println("=== go.mod 檔案檢查 ===")
    
    // 取得當前工作目錄
    pwd, _ := os.Getwd()
    fmt.Printf("當前目錄: %s\n", pwd)
    
    // 尋找 go.mod 檔案（向上搜尋）
    goModPath := findGoMod(pwd)
    if goModPath == "" {
        fmt.Println("\n❌ 沒有找到 go.mod 檔案")
        fmt.Println("💡 這不是一個 Go Module 專案")
        fmt.Println("💡 執行 'go mod init <module-name>' 來建立")
        return
    }
    
    fmt.Printf("\n✅ 找到 go.mod: %s\n", goModPath)
    
    // 讀取 go.mod 內容
    content, err := ioutil.ReadFile(goModPath)
    if err != nil {
        fmt.Printf("❌ 無法讀取 go.mod: %v\n", err)
        return
    }
    
    fmt.Println("\n📄 go.mod 內容:")
    fmt.Println(strings.Repeat("-", 50))
    fmt.Println(string(content))
    fmt.Println(strings.Repeat("-", 50))
    
    // 分析 go.mod 內容
    lines := strings.Split(string(content), "\n")
    fmt.Println("\n🔍 go.mod 分析:")
    
    for _, line := range lines {
        line = strings.TrimSpace(line)
        if line == "" || strings.HasPrefix(line, "//") {
            continue
        }
        
        switch {
        case strings.HasPrefix(line, "module "):
            fmt.Printf("  📦 模組名稱: %s\n", strings.TrimPrefix(line, "module "))
        case strings.HasPrefix(line, "go "):
            fmt.Printf("  🏷️  Go 版本: %s\n", strings.TrimPrefix(line, "go "))
        case strings.HasPrefix(line, "require ("):
            fmt.Println("  📋 開始依賴區塊")
        case strings.Contains(line, "// indirect"):
            fmt.Printf("  ↪️  間接依賴: %s\n", strings.Split(line, " // indirect")[0])
        case strings.Contains(line, " v") && !strings.Contains(line, "//"):
            fmt.Printf("  📦 直接依賴: %s\n", line)
        }
    }
}

// findGoMod 向上搜尋 go.mod 檔案
func findGoMod(dir string) string {
    for {
        goModPath := filepath.Join(dir, "go.mod")
        if _, err := os.Stat(goModPath); err == nil {
            return goModPath
        }
        
        parent := filepath.Dir(dir)
        if parent == dir {
            break  // 已到根目錄
        }
        dir = parent
    }
    return ""
}

## 4. go.sum 檔案與 Checksum 驗證機制（Go 1.13+）

Go 1.13 引入了 `go.sum` 檔案和 sumdb（校驗和資料庫），用於確保模組的完整性和安全性。

### 🔐 go.sum 的安全保證

`go.sum` 檔案記錄每個依賴模組的 SHA-256 校驗和，確保：
- **完整性驗證**: 檢測模組是否被篡改
- **可重現建置**: 保證不同環境得到相同結果
- **供應鏈安全**: 防止惡意模組注入

### 📋 go.sum 檔案格式

```
github.com/gin-gonic/gin v1.9.1 h1:4idEAncQnU5cB7BeOkPtxjfCSye0AAm1R0RVIqJ+Jmg=
github.com/gin-gonic/gin v1.9.1/go.mod h1:hPrL7YrpYKXt5YId3A/Tnip5kqbEAP+KLuI3SUcPTeU=
github.com/stretchr/testify v1.8.4 h1:CcVxjf3Q3iL9o5VqKHgXKm6Vg9h1i7v7T3Y3fB4q1JDYZ==
github.com/stretchr/testify v1.8.4/go.mod h1:sz/lmYIOXD/1dqDkqEQP7T5cUFXQDJSZCDJ6KdBMF5s=
```

**格式說明**：
- **第一行**: `模組@版本 h1:整個模組zip的SHA-256`
- **第二行**: `模組@版本/go.mod h1:go.mod檔案的SHA-256`

### 🌐 Sum Database（sumdb）機制

Go 1.13+ 使用 Google 維護的公開校驗和資料庫：
- **預設 sumdb**: `sum.golang.org`
- **功能**: 提供全球一致的模組校驗和
- **快取位置**: `$GOPATH/pkg/sumdb/`

### 🔄 go.sum 生成與更新時機

以下操作會更新 `go.sum`：
- `go get` - 添加新依賴時
- `go mod download` - 下載依賴時
- `go build` / `go run` - 首次解析依賴時
- `go mod tidy` - 清理依賴時

In [ ]:
/* 檢查 go.sum 檔案與校驗和機制 */
package main

import (
    "fmt"
    "io/ioutil"
    "os"
    "path/filepath"
    "strings"
)

func main() {
    fmt.Println("=== go.sum 校驗和檔案檢查 ===")
    
    // 取得當前目錄
    pwd, _ := os.Getwd()
    
    // 尋找 go.sum 檔案
    goSumPath := findGoSum(pwd)
    if goSumPath == "" {
        fmt.Println("❌ 沒有找到 go.sum 檔案")
        fmt.Println("💡 當專案有依賴時，會自動產生 go.sum")
        fmt.Println("💡 執行 'go get <package>' 或 'go mod tidy' 來生成")
        return
    }
    
    fmt.Printf("✅ 找到 go.sum: %s\n", goSumPath)
    
    // 讀取 go.sum 內容
    content, err := ioutil.ReadFile(goSumPath)
    if err != nil {
        fmt.Printf("❌ 無法讀取 go.sum: %v\n", err)
        return
    }
    
    lines := strings.Split(strings.TrimSpace(string(content)), "\n")
    if len(lines) == 0 || (len(lines) == 1 && lines[0] == "") {
        fmt.Println("📄 go.sum 檔案是空的")
        return
    }
    
    fmt.Printf("\n📊 go.sum 統計:\n")
    fmt.Printf("  總校驗和記錄: %d 筆\n", len(lines))
    
    // 分析 go.sum 內容
    moduleCount := 0
    goModCount := 0
    modules := make(map[string]bool)
    
    fmt.Println("\n🔍 校驗和記錄分析:")
    for i, line := range lines {
        if line == "" {
            continue
        }
        
        parts := strings.Fields(line)
        if len(parts) >= 3 {
            module := parts[0]
            version := parts[1]
            hash := parts[2]
            
            modules[module] = true
            
            if strings.Contains(version, "/go.mod") {
                goModCount++
                if i < 5 {  // 只顯示前 5 筆
                    fmt.Printf("  📄 %s %s\n", module, version)
                    fmt.Printf("      校驗和: %s...\n", hash[:20])  // 截短顯示
                }
            } else {
                moduleCount++
                if i < 5 {  // 只顯示前 5 筆
                    fmt.Printf("  📦 %s %s\n", module, version)
                    fmt.Printf("      校驗和: %s...\n", hash[:20])  // 截短顯示
                }
            }
        }
        
        if i >= 5 {
            fmt.Printf("  ... 還有 %d 筆記錄\n", len(lines)-5)
            break
        }
    }
    
    fmt.Printf("\n📈 統計摘要:\n")
    fmt.Printf("  不同模組數: %d\n", len(modules))
    fmt.Printf("  模組 zip 校驗和: %d 筆\n", moduleCount)
    fmt.Printf("  go.mod 校驗和: %d 筆\n", goModCount)
    
    fmt.Println("\n🔐 安全提醒:")
    fmt.Println("  • go.sum 確保模組完整性")
    fmt.Println("  • 每個依賴都有兩筆校驗和記錄")
    fmt.Println("  • 不要手動修改 go.sum 檔案")
    fmt.Println("  • 提交程式碼時應包含 go.sum")
}

// findGoSum 向上搜尋 go.sum 檔案
func findGoSum(dir string) string {
    for {
        goSumPath := filepath.Join(dir, "go.sum")
        if _, err := os.Stat(goSumPath); err == nil {
            return goSumPath
        }
        
        parent := filepath.Dir(dir)
        if parent == dir {
            break
        }
        dir = parent
    }
    return ""
}

## 5. Go 相關命令行為完整對照表

不同 Go 版本中，各命令的行為有顯著差異。以下是完整的對照表：

### 📊 Go 命令對照表

| 命令 | 作用 | 對檔案/資料的影響 | 版本差異 | 備註 |
|------|------|------------------|----------|------|
| `go run main.go` | 編譯並直接執行 | 不產生 binary；使用 GOMODCACHE 依賴 | 無 | 適合快速執行程式 |
| `go build` | 編譯成 binary | binary 生成於當前目錄；依賴從 GOMODCACHE 讀取 | 無 | 不修改 go.mod/go.sum |
| `go install` | 編譯並安裝 binary | binary 生成於 `$GOBIN`/`$GOPATH/bin` | **Go 1.17+** 可用 `pkg@version` | 不修改專案 go.mod |
| `go get pkg@ver` | 新增/更新依賴版本 | 修改 go.mod & go.sum；下載依賴到 GOMODCACHE | **Go 1.17+** 不再安裝到 GOBIN | 主要用於更新依賴 |
| `go mod download` | 只下載依賴到本地快取 | 不修改 go.mod；可能驗證 checksum | **Go 1.13+** 加入 sumdb 驗證 | 適合 CI/CD 預下載 |
| `go mod tidy` | 清理並補齊依賴 | 刪除未使用依賴、補齊缺失依賴；更新 go.sum | **Go 1.17+** 改進演算法 | 保持專案依賴乾淨 |
| `go env` | 顯示環境變數 | 不修改任何檔案 | 新版本加入更多環境變數 | 查看 GOPATH, GOMODCACHE, GOBIN |

### 🔄 版本關鍵差異說明

#### Go 1.16 之前 vs 之後

**Go 1.16 之前**：
```bash
go get github.com/gin-gonic/gin@latest  # 下載並安裝到 GOBIN
```

**Go 1.17+**：
```bash
go get github.com/gin-gonic/gin@latest  # 只更新 go.mod，不安裝
go install github.com/gin-gonic/gin@latest  # 安裝到 GOBIN
```

#### GO111MODULE 環境變數演進

- **Go 1.11-1.12**: 預設 `auto`，在 GOPATH 內使用舊模式
- **Go 1.13-1.15**: 預設 `auto`，但偏好 module 模式
- **Go 1.16+**: 移除 `auto` 支援，強制使用 module 模式

In [ ]:
/* 示範各種 Go 命令的使用與效果 */
package main

import (
    "fmt"
    "os"
    "os/exec"
    "runtime"
    "strings"
)

func main() {
    fmt.Println("=== Go 命令使用示範 ===")
    fmt.Printf("Go 版本: %s\n\n", runtime.Version())
    
    // 常用的 go env 變數
    envVars := []string{"GOROOT", "GOPATH", "GOMODCACHE", "GOPROXY", "GOSUMDB"}
    
    fmt.Println("🔧 重要環境變數:")
    for _, envVar := range envVars {
        if value := getGoEnv(envVar); value != "" {
            fmt.Printf("  %s: %s\n", envVar, value)
        }
    }
    
    fmt.Println("\n📋 常用 Go 命令說明:")
    
    commands := []struct {
        cmd   string
        desc  string
        usage string
    }{
        {"go run", "編譯並執行程式", "go run main.go"},
        {"go build", "編譯程式成執行檔", "go build -o myapp"},
        {"go install", "編譯並安裝到 GOBIN", "go install github.com/tool@latest"},
        {"go get", "下載並更新依賴", "go get github.com/gin-gonic/gin@v1.9.1"},
        {"go mod tidy", "整理模組依賴", "go mod tidy"},
        {"go mod download", "下載依賴到快取", "go mod download"},
        {"go mod init", "初始化新模組", "go mod init example.com/myproject"},
        {"go mod verify", "驗證依賴完整性", "go mod verify"},
    }
    
    for _, cmd := range commands {
        fmt.Printf("\n  🚀 %s\n", cmd.cmd)
        fmt.Printf("     說明: %s\n", cmd.desc)
        fmt.Printf("     用法: %s\n", cmd.usage)
    }
    
    // 版本差異提醒
    fmt.Println("\n⚠️  版本差異提醒:")
    fmt.Println("  • Go 1.17+: go get 不再自動安裝，需要用 go install")
    fmt.Println("  • Go 1.16+: 不再支援 GOPATH 模式 (GO111MODULE=auto)")
    fmt.Println("  • Go 1.13+: go.sum 和 sumdb 校驗和驗證")
    
    fmt.Println("\n💡 最佳實踐:")
    fmt.Println("  1. 使用 go mod init 建立新專案")
    fmt.Println("  2. 定期執行 go mod tidy 清理依賴")
    fmt.Println("  3. 提交程式碼時包含 go.mod 和 go.sum")
    fmt.Println("  4. 使用 go install 安裝工具程式")
}

// getGoEnv 取得 go env 變數值
func getGoEnv(name string) string {
    cmd := exec.Command("go", "env", name)
    output, err := cmd.Output()
    if err != nil {
        return ""
    }
    return strings.TrimSpace(string(output))
}

## 6. 命令與檔案流向總結

理解 Go 命令如何與不同檔案和目錄互動，是掌握 Go 開發流程的關鍵。

### 🔄 完整的檔案流向圖

```
你的專案 (有 go.mod)
├─ go build / go run
│   └─ 讀取依賴 → GOMODCACHE/pkg/mod
├─ go install
│   └─ 編譯輸出 → GOBIN / GOPATH/bin
├─ go get / go mod tidy
│   ├─ 修改 → go.mod / go.sum
│   └─ 下載依賴 → GOMODCACHE
├─ go mod download
│   ├─ 下載依賴 → GOMODCACHE
│   └─ 驗證校驗和 → sumdb
└─ go mod verify
    └─ 檢查校驗和 → go.sum vs GOMODCACHE
```

### 📊 依賴解析流程

1. **讀取 go.mod** → 確定需要的依賴版本
2. **檢查 GOMODCACHE** → 確認依賴是否已下載
3. **下載缺失依賴** → 從 GOPROXY 下載到 GOMODCACHE
4. **驗證校驗和** → 對照 go.sum 或查詢 sumdb
5. **編譯連結** → 使用 GOMODCACHE 中的依賴

### 🎯 不同命令的檔案操作

#### 只讀取，不修改
- `go build` / `go run` / `go test`
- `go mod verify` / `go mod graph`

#### 修改 go.mod/go.sum
- `go get` (添加/更新依賴)
- `go mod tidy` (清理依賴)
- 首次 `go build` (解析缺失依賴)

#### 只下載，不修改專案檔案
- `go mod download`
- `go install pkg@version`

In [ ]:
/* 示範完整的 Go 專案工作流程 */
package main

import (
    "fmt"
    "os"
    "path/filepath"
)

func main() {
    fmt.Println("=== Go 專案工作流程示範 ===")
    
    // 取得當前目錄資訊
    pwd, _ := os.Getwd()
    fmt.Printf("當前專案目錄: %s\n\n", pwd)
    
    // 檢查專案狀態
    fmt.Println("🔍 專案檔案檢查:")
    checkProjectFiles(pwd)
    
    fmt.Println("\n📋 建議的工作流程:")
    
    workflows := []struct {
        step    string
        command string
        purpose string
        effect  string
    }{
        {
            "1. 初始化專案",
            "go mod init example.com/myproject",
            "建立 go.mod 檔案",
            "建立模組定義檔",
        },
        {
            "2. 添加依賴",
            "go get github.com/gin-gonic/gin@v1.9.1",
            "安裝特定版本依賴",
            "更新 go.mod 和 go.sum",
        },
        {
            "3. 開發測試",
            "go run main.go",
            "快速執行程式測試",
            "臨時編譯執行",
        },
        {
            "4. 整理依賴",
            "go mod tidy",
            "清理未使用的依賴",
            "優化 go.mod 和 go.sum",
        },
        {
            "5. 建置程式",
            "go build -o myapp",
            "編譯成執行檔",
            "產生 binary 檔案",
        },
        {
            "6. 安裝工具",
            "go install",
            "安裝到 GOBIN",
            "全域可用的執行檔",
        },
    }
    
    for _, wf := range workflows {
        fmt.Printf("\n  %s\n", wf.step)
        fmt.Printf("    命令: %s\n", wf.command)
        fmt.Printf("    目的: %s\n", wf.purpose)
        fmt.Printf("    效果: %s\n", wf.effect)
    }
    
    fmt.Println("\n🚀 進階工作流程:")
    
    advanced := []string{
        "go mod download  # 預先下載所有依賴（CI/CD 用）",
        "go mod verify    # 驗證依賴完整性",
        "go mod graph     # 查看依賴關係圖",
        "go mod why <pkg> # 查看為什麼需要某個依賴",
        "go clean -modcache # 清理模組快取",
    }
    
    for _, cmd := range advanced {
        fmt.Printf("  • %s\n", cmd)
    }
    
    fmt.Println("\n💡 最佳實踐提醒:")
    fmt.Println("  ✅ 定期執行 go mod tidy")
    fmt.Println("  ✅ 提交程式碼包含 go.sum")
    fmt.Println("  ✅ 使用語義化版本控制")
    fmt.Println("  ✅ 避免手動修改 go.mod")
}

// checkProjectFiles 檢查專案檔案狀態
func checkProjectFiles(dir string) {
    files := []struct {
        name string
        desc string
    }{
        {"go.mod", "模組定義檔"},
        {"go.sum", "依賴校驗和檔案"},
        {"main.go", "主程式檔案"},
        {".gitignore", "Git 忽略檔案"},
    }
    
    for _, file := range files {
        path := filepath.Join(dir, file.name)
        if _, err := os.Stat(path); err == nil {
            fmt.Printf("  ✅ %s - %s\n", file.name, file.desc)
        } else {
            fmt.Printf("  ❌ %s - %s (不存在)\n", file.name, file.desc)
        }
    }
}

## 7. 從 GOPATH 遷移到 Go Modules

如果你有舊的 GOPATH 專案，以下是遷移到 Go Modules 的步驟：

### 🔄 遷移步驟

1. **複製專案到 GOPATH 外**
   ```bash
   cp -r $GOPATH/src/github.com/user/project ~/projects/project
   cd ~/projects/project
   ```

2. **初始化模組**
   ```bash
   go mod init github.com/user/project
   ```

3. **解析依賴**
   ```bash
   go mod tidy
   ```

4. **測試編譯**
   ```bash
   go build
   go test ./...
   ```

### ⚠️ 常見遷移問題

- **相對導入**: 將 `import "./subpackage"` 改為完整模組路徑
- **vendor 目錄**: 舊的 vendor 可以刪除，go.mod 會管理依賴
- **版本衝突**: 可能需要手動解決不相容的依賴版本

## 8. 不同 Go 版本的相容性注意事項

### 📋 版本相容性矩陣

| 功能 | Go 1.11 | Go 1.13 | Go 1.16 | Go 1.17+ | Go 1.20+ |
|------|---------|---------|---------|----------|----------|
| **GOPATH 模式** | 支援 | 支援 | 棄用 | 移除 | 移除 |
| **Go Modules** | 實驗性 | 預設 | 強制 | 強制 | 強制 |
| **go.sum** | ❌ | ✅ | ✅ | ✅ | ✅ |
| **sumdb** | ❌ | ✅ | ✅ | ✅ | ✅ |
| **go install pkg@ver** | ❌ | ❌ | ✅ | ✅ | ✅ |
| **go get 安裝行為** | 安裝 | 安裝 | 安裝 | 不安裝 | 不安裝 |

### 🎯 升級建議

- **Go 1.20+**: 推薦用於新專案，效能和安全性最佳
- **Go 1.17+**: 最低建議版本，完整的 module 支援
- **Go 1.16-**: 不建議新專案使用

### 🔧 環境設定建議

```bash
# 現代 Go 開發環境設定
export GO111MODULE=on          # 明確啟用 module 模式
export GOPROXY=direct          # 或使用 goproxy.cn 等加速服務
export GOSUMDB=sum.golang.org  # 使用官方校驗和資料庫
```

## 章節總結

### 🎯 重點回顧

1. **Go 版本演進**：從 GOPATH 模式 (Go 1.11 前) 到 Go Modules (Go 1.11+)
2. **依賴管理**：go.mod 定義依賴，go.sum 保證安全性
3. **命令差異**：Go 1.17+ 中 go get 和 go install 職責分離
4. **最佳實踐**：使用 Go Modules，定期 go mod tidy，提交 go.sum

### 📚 下一步學習

- 第二章：Go 基本語法與型別系統
- 實際動手建立一個 Go Module 專案
- 練習使用不同的 go 命令

### 🔗 有用的資源

- [Go Modules 官方文件](https://golang.org/ref/mod)
- [Go 1.17 發行說明](https://golang.org/doc/go1.17)
- [依賴管理最佳實踐](https://golang.org/doc/modules/managing-dependencies)